<h1>***Part 1***</h1>

<h2>#1 We will get the information about neighborhoods of Toronto from Wikipedia link and create a dataframe with it. The Wikipedia page that we will scrape is <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">here</a>.</h2>

In [59]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all of the anchor tags
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [62]:
#Let's see the shape of our dataframe.
print(df.shape)

(103, 3)


In [66]:
#Let's create a list of postal codes...
postal_codes=[]
for dic in table_contents:
    postal_codes.append(dic['PostalCode'])
print(postal_codes)

['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B', 'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C', 'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H', 'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R', 'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S', 'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V', 'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X', 'M4Y', 'M7Y', 'M8Y', 'M8Z']


<h1>***Part 2***</h1>

<h2>#2 We will get the coordinate information of neighborhoods of Toronto from Google locations API and then we will add them to our dataframe. postal_codes list will be used here.</h2>

In [73]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'
# https://developers.google.com/maps/documentation/geocoding/intro

if api_key is False:
    api_key = 42
    serviceurl = 'http://py4e-data.dr-chuck.net/json?'
else :
    serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

latitude=[]
longitude=[]

for i in postal_codes:
    address = ('Toronto, Ontario ,', i)
    if len(address) < 1: break

    parms = dict()
    parms['address'] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        print(data)
        continue

    lat = js['results'][0]['geometry']['location']['lat']
    lng = js['results'][0]['geometry']['location']['lng']
    print('lat', lat, 'lng', lng)
    latitude.append(lat)
    longitude.append(lng)
print(latitude)
print(longitude)

Retrieving http://py4e-data.dr-chuck.net/json?address=%28%27Toronto%2C+Ontario+%2C%27%2C+%27M3A%27%29&key=42
Retrieved 2155 characters
lat 43.7394218 lng -79.3241424
Retrieving http://py4e-data.dr-chuck.net/json?address=%28%27Toronto%2C+Ontario+%2C%27%2C+%27M4A%27%29&key=42
Retrieved 2283 characters
lat 43.7173705 lng -79.3055268
Retrieving http://py4e-data.dr-chuck.net/json?address=%28%27Toronto%2C+Ontario+%2C%27%2C+%27M5A%27%29&key=42
Retrieved 2242 characters
lat 43.6502783 lng -79.35911039999999
Retrieving http://py4e-data.dr-chuck.net/json?address=%28%27Toronto%2C+Ontario+%2C%27%2C+%27M6A%27%29&key=42
Retrieved 2172 characters
lat 43.725435 lng -79.4521692
Retrieving http://py4e-data.dr-chuck.net/json?address=%28%27Toronto%2C+Ontario+%2C%27%2C+%27M7A%27%29&key=42
Retrieved 2196 characters
lat 43.66227809999999 lng -79.3915272
Retrieving http://py4e-data.dr-chuck.net/json?address=%28%27Toronto%2C+Ontario+%2C%27%2C+%27M9A%27%29&key=42
Retrieved 2153 characters
lat 43.6639296 lng -79

In [74]:
print(len(latitude))
print(len(longitude))

103
103


<p>#3 We succesfully retreived the latitude and longitude information from API. Let's put them into our dataframe.</p>

In [75]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.739422,-79.324142
1,M4A,North York,Victoria Village,43.717371,-79.305527
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650278,-79.359110
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.725435,-79.452169
4,M7A,Queen's Park,Ontario Provincial Government,43.662278,-79.391527


In [87]:
df_postcode = df.groupby(['PostalCode', 'Borough'])
df_postcode.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.739422,-79.324142
1,M4A,North York,Victoria Village,43.717371,-79.305527
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650278,-79.359110
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.725435,-79.452169
4,M7A,Queen's Park,Ontario Provincial Government,43.662278,-79.391527
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.645706,-79.521070
99,M4Y,Downtown Toronto,Church and Wellesley,43.653226,-79.383184
100,M7Y,East Toronto Business,Enclave of M4L,43.653226,-79.383184
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.623943,-79.485493


<h1>***Part 3***</h1>

<h2>#4 Let's explore and cluster the neighborhoods in Toronto.</h2>

In [96]:
import numpy as np
import pandas as pd
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium

import requests
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Done...')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Caglar\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         102 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0




geographiclib-1.52   | 35 KB     |            |   0% 
geographiclib-1.52   | 35 KB     | ####5      |  46% 
geographiclib-1.52   | 35 KB     | ########## | 100% 

geopy-2.2.0          | 67 KB     |            |   0% 
geopy-2.2.0

<h2>Note : Let's find the boroughs that contain the word Toronto.</h2>

In [106]:
toronto_data = df[df['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650278,-79.359110
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.656081,-79.380171
2,M5C,Downtown Toronto,St. James Town,43.650480,-79.377370
3,M4E,East Toronto,The Beaches,43.678594,-79.294004
4,M5E,Downtown Toronto,Berczy Park,43.648688,-79.371545


In [107]:
#Toronto's coordinates from google,
Toronto_latitude = 43.6532
Toronto_longitude= -79.3832

# Let's create a map of TORONTO:
map_toronto = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

# Markers from our data
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [145]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: PJBTRT00FYRY1WLAAECF5HZRGLO14LV3QODCIB0FUUIPD2PJ
CLIENT_SECRET:LRSVQHQMBFHM3R0DR5QC35X0MIJGGEAWCCR0QVJZ0VNQF4EB


In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h2>Explore places around Toronto.</h2>

In [111]:
toronto_places = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])
print(toronto_places.shape)

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [112]:
toronto_places.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650278,-79.35911,The Distillery Historic District,43.650244,-79.359323,Historic Site
1,"Regent Park, Harbourfront",43.650278,-79.35911,Arvo,43.649963,-79.361442,Coffee Shop
2,"Regent Park, Harbourfront",43.650278,-79.35911,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650278,-79.35911,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
4,"Regent Park, Harbourfront",43.650278,-79.35911,Balzac's Coffee,43.649797,-79.359142,Coffee Shop


In [114]:
toronto_places.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",9,9,9,9,9,9
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58
Central Bay Street,92,92,92,92,92,92
Christie,82,82,82,82,82,82
Church and Wellesley,92,92,92,92,92,92
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26
Davisville North,95,95,95,95,95,95


<h2>Analyzing each neigborhood.</h2>

In [115]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_places[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_places['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
toronto_onehot.shape

(2380, 244)

In [118]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Berczy Park,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000
3,Central Bay Street,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,...,0.000000,0.021739,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000
4,Christie,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,...,0.000000,0.012195,0.000000,0.024390,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,...,0.000000,0.021739,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Commerce Court, Victoria Hotel",0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021053,0.000000,...,0.000000,0.010526,0.010526,0.000000,0.010526,0.000000,0.010526,0.000000,0.010526,0.000000
9,"Dufferin, Dovercourt Village",0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.028571,0.000000,0.000000


In [119]:
toronto_grouped.shape

(39, 244)

In [121]:
#Each neighborhood along with the top 5 most common places.
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2          Restaurant  0.04
3                 Pub  0.03
4                Café  0.03


----Brockton, Parkdale Village, Exhibition Place----
                  venue  freq
0           Music Venue  0.22
1    Athletics & Sports  0.11
2         Movie Theater  0.11
3  Fast Food Restaurant  0.11
4                Circus  0.11


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
            venue  freq
0     Pizza Place  0.07
1           Hotel  0.07
2             Bar  0.07
3     Coffee Shop  0.07
4  Scenic Lookout  0.05


----Central Bay Street----
            venue  freq
0  Clothing Store  0.10
1     Coffee Shop  0.07
2           Hotel  0.05
3      Restaurant  0.03
4  Cosmetics Shop  0.02


----Christie----
                 venue  freq
0    Korean Restaurant  0.17
1          Coffee Shop  0.06
2        Grocery Store  0.05
3  Japan

<h2>Let's put this data into pandas dataframe.</h2>

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [124]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Italian Restaurant,Restaurant,Pub,Café,Seafood Restaurant,Cocktail Bar,Japanese Restaurant,Bakery,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Music Venue,Athletics & Sports,Movie Theater,Fast Food Restaurant,Circus,Lounge,Theme Park,Park,Nail Salon,Noodle House
2,"CN Tower, King and Spadina, Railway Lands, Har...",Pizza Place,Hotel,Bar,Coffee Shop,Scenic Lookout,Baseball Stadium,Gym,Sports Bar,Plaza,Supermarket
3,Central Bay Street,Clothing Store,Coffee Shop,Hotel,Restaurant,Cosmetics Shop,American Restaurant,Vegetarian / Vegan Restaurant,New American Restaurant,Breakfast Spot,Diner
4,Christie,Korean Restaurant,Coffee Shop,Grocery Store,Japanese Restaurant,Mexican Restaurant,Bakery,Video Store,Pub,Dessert Shop,Ice Cream Shop


<h2>Cluster Neighbourhoods</h2>

In [126]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 2, 2, 3, 2, 2, 3, 3, 3])

In [135]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650278,-79.359110,3,Coffee Shop,Bakery,Boat or Ferry,Theater,Tech Startup,Chocolate Shop,Gym Pool,Breakfast Spot,Gym / Fitness Center,Mexican Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.656081,-79.380171,2,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Department Store,Fast Food Restaurant,Lingerie Store,Sushi Restaurant,Bubble Tea Shop
2,M5C,Downtown Toronto,St. James Town,43.650480,-79.377370,2,Coffee Shop,Restaurant,Café,Gym,Seafood Restaurant,Cosmetics Shop,American Restaurant,Clothing Store,Gastropub,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.678594,-79.294004,3,Pub,Bakery,Gastropub,Health Food Store,Pizza Place,Trail,Asian Restaurant,Park,Coffee Shop,Nightclub
4,M5E,Downtown Toronto,Berczy Park,43.648688,-79.371545,3,Coffee Shop,Italian Restaurant,Restaurant,Pub,Café,Seafood Restaurant,Cocktail Bar,Japanese Restaurant,Bakery,Breakfast Spot


In [143]:
# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters